## Analyze A/B Test Results
### by Ibrahim Olayiwola

## Table of contents.

- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)
- [Part IV - Conclusions](#conclusions)


<a id='intro'></a>
### Introduction
>A/B tests are very commonly performed by to test weather a changeversion of a webpage brings about more conversion in users.
For this project, I will be working to understand the results of an A/B test run by an e-commerce website. The goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.

>The A/B test has been carried out and what remains is for me to analyze it. The reason I'm doing this analysis is to help an e-commerce company know if they should implement a new page, keep the old page, or perhaps run the experiment longer to make a decision. The A/B test results is save in a csv file name ab_data.


<a id='probability'></a>
### Part I - Probability

>In this part, we'd calculate some probabilities, but to get started, we'd import our libraries and do some preliminary cleaning of the data. We'd also copy our data to use for the analysis to preserve our original dataset.

In [1]:
import pandas as pd
import numpy as np
import random 
import matplotlib.pyplot as plt

# Setting seed so as to get uniform answer when another analyst carries out this test.
random.seed(42)

>Now, the data will be read in and stored in a dataframe.

In [2]:
df = pd.read_csv('ab_data.csv')

# Checking the first 5 rows of the dataframe.
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


>From the above rows, we can see that;
- There are 5 columns in the dataset.
- The users were divided into two groups, the control group which were shown the old landing page and the treatment group which were shown the new landing page.
- There is also a column that shows if a user was converted or not based on the page they were shown.

Next, I would explore the dataset to know more about it.

In [3]:
# Number of rows in the dataset
df.shape[0]

294478

>There are two hundred and ninety-four thousand, four hundred and seventy-eight (294,478) rows in the dataset.

>Next is to check the number of unique users and other unique details about the dataset.

In [4]:
# Unique details
df.nunique()

user_id         290584
timestamp       294478
group                2
landing_page         2
converted            2
dtype: int64

> So, from the output above we can say that
- Out of 294478 users, 290584 are unique meaning 3894 are duplicate of some users.
- The users were divided into two main groups, the control group and the treatment group.
- There are two landing pages, the old page and the new page.
- Users were either converted or not.

Now, we'd check the proportion of users converted.

In [5]:
# Proportion of users converted
df.converted.mean()

0.11965919355605512

> Approximately 12% of users were converted. 

> From basic knowledge and from the data thats been previewed, we know that the treatment group is meant to match the new landing page and the control group is meant to match the old landing page. However, there might be some errors so we would check if some users in a group doesn't match the page they were supposed to match.

In [6]:
# The number of times the new_page and treatment don't match.
df.query('(group != "treatment" and landing_page == "new_page") or (group == "treatment" and landing_page != "new_page")').user_id.count()

3893

> The number of time the pages and the landing page doesn't match is 3893.

>Now, we would check for missing values in the dataset.

In [7]:
# Checking for missing values.
df.isna().any()

user_id         False
timestamp       False
group           False
landing_page    False
converted       False
dtype: bool

> From the above, there are no missing values in the dataset, and since we're not sure about the 3893 pages and group that does not match, we would drop the rows which the values does not match.

In [8]:
# Dropping rows in which the treatment does not match the landing page and save in df2.
df_c = df.query('group == "control" and landing_page == "old_page"')
df_t = df.query('group == "treatment" and landing_page == "new_page"')
df2 = df_c.append(df_t)
df2.shape[0]

290585

>After dropping, the number of rows decreased to 290585. Next, we'd double check for values that don't match and check for duplicate values.

In [9]:
# Double checking for pages and group that doesn't match.
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

0

In [10]:
# Check for duplicate rows
# Checking for duplicates user_id
df2[df2.duplicated(['user_id'])]

,user_id,timestamp,group,landing_page,converted
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


>There is a duplicate row of user_id 773192. Let's check for more information on the duplicate.

In [11]:
df2.query('user_id == "773192"')

,user_id,timestamp,group,landing_page,converted
1899,773192,2017-01-09 05:37:58.781806,treatment,new_page,0
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


The duplicated row is 1899 and it has almost same information except for the timestamp. One row should be removed.

In [12]:
# Dropping row 1899
df2 = df2.drop([1899])

In [13]:
# Confirming the row has been drop
print(df2.info())
df2[df2.duplicated(['user_id'])]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290584 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       290584 non-null  int64 
 1   timestamp     290584 non-null  object
 2   group         290584 non-null  object
 3   landing_page  290584 non-null  object
 4   converted     290584 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 13.3+ MB
None


,user_id,timestamp,group,landing_page,converted


>From the above cell, we can see that there are 290584 rows and that there are no duplicated user_id.

>Alright, now we're ready to calculate the probabilities. We're going to check for the probability of
- A user converting regardless of the page they recieved.
- A user converting when he/she is shown the old landing page(i.e In the control group).
- A user converting when he/she is shown the new landing page (ie In the treatment group).
- A user being shown the new landing page and the old landing page.

**Probability of a user converting regardless of page shown.**

In [14]:
# Probability of conversion
prob_conv = df2.converted.mean()
prob_conv

0.11959708724499628

>The probability of a user converting is approximately **12%**

>Our next calculation is the probability of conversion when in the control group, ie he/she is shown the old landing page.

**Probability of a user converting when in control group**

In [15]:
control_prob_conv = df2.query('group == "control"')['converted'].mean()
control_prob_conv

0.1203863045004612

> From the immediate cell above, we got that the probability of a user being converted when in the control group, shown the old landing page, is also approximately **12%**. Based on this and the probability of the overall conversion being 12%, it is safe to assume that the probability of a user converting when in the treatment group is also approximately 12%. However let's check our assumptions.

**Probability of a user converting when in treatment group**

In [16]:
treat_prob_conv = df2.query('group == "treatment"')['converted'].mean()
treat_prob_conv

0.11880806551510564

> We got our assumption right in that the probability of a user converting while in the treatment group is approximately **12%**. However, before we can claim that a user will convert regardless of the group is 12%, we need to know the probability of a user being placed in the control group and the probability of being placed in the treatment group.

**Probability of a user being placed in the control group**

In [17]:
prob_cont_grp = df2.query('group == "control"')['group'].count()/df2.shape[0]
prob_cont_grp

0.4999380557773312

In [18]:
prob_treat_grp = 1 - prob_cont_grp
prob_treat_grp

0.5000619442226688

>The probability of being placed in one of the groups in approximately 0.5 which is fair and not biased. Therefore we can say that there is no significant change in the conversion rate if the new landing page is adopted. 

In the next part, we'd run a hypothesis test with a type 1 error rate of 5%.

<a id='ab_test'></a>
### Part II - A/B Test